In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [29]:
def plot_metric(history, metric):
  plt.plot(history.history[metric], label='Training data')
  plt.plot(history.history['val_' + metric], label='Validation data')
  plt.title(metric)
  plt.ylabel(metric)
  plt.xlabel('Epoch')
  plt.legend(loc="upper right")
  plt.show()

def open_image(archivo):
    datos = np.array(plt.imread(archivo))
    sh=np.shape(datos)
    datos2 = []
    colores = []
    for i in range(sh[0]):
        for j in range(sh[1]):
            color = datos[i][j][0] + datos[i][j][1] * 255 + datos[i][j][2] * 255 * 255

            if(color != 16646655):
                if not (color in colores):
                    colores.append(color)
                color = colores.index(color)

                datos2.append([j, 1000 - i, color])

    return np.array(datos2)

def plot_image(P, T):
    marcadores = {0:('+','b'), 1:('o','g'), 2:('x', 'y'), 3:('*', 'm'), 4:('.', 'r'), 5:('+', 'k')}

    plt.clf()
    gs = gridspec.GridSpec(1, 1)
    ax = plt.subplot(gs[0, 0])

    # Muestras
    for class_value in np.unique(T):
        x = []
        y = []
        for i in range(len(T)):
            if T[i] == class_value:
                x.append(P[i, 0])
                y.append(P[i, 1])
        ax.scatter(x, y, marker=marcadores[class_value][0], color=marcadores[class_value][1])


In [ ]:
file_name = "" # Nombre del archivo con la imagen a analizar

X = open_image(file_name)

X = np.random.permutation(X)
T = X[:, 2]    # Variable objetivo (Clase de 0 a N)
P = X[:, 0:2]  # Variables de entrada (X y Y de los píxeles no-blancos de la imagen)

# Número de clases del problema (Cantidad de colores no-blancos de la imagen)
nclases = max(T) + 1

# Crear la salida con codificación one-hot
T_OH = [T==0]
for i in range(1,nclases):
  T_OH = np.concatenate( (T_OH, [T==i]), axis=0)
T_OH = T_OH.astype(int).T

# Escalar la entrada a valores [0..1]
column_min_values = np.min(P, axis=0)
P = (P - column_min_values) / (np.max(P, axis=0) - column_min_values)

# Plotear la imagen
plot_image(P, T)

In [ ]:
# Crear una red que sea capaz de resolver el problema

d_in = 2  # Número de variables de entrada. Para este ejercicio debe ser 2 ya que
          # el dataset está formado por dos variables: el valor X y Y de cada pixel
          # no-blanco de la imagen
d_out = nclases # Número de salidas. La misma que la cantidad de clases (usando one_hot)

modelo = keras.Sequential([
    keras.layers.Dense(5, input_shape=(d_in,), activation="tanh"),
    keras.layers.Dense(d_out, activation="softmax")
	])

modelo.compile(
  optimizer = keras.optimizers.SGD(learning_rate=0.1),
  loss = 'categorical_crossentropy', metrics = ['accuracy']
)

modelo.summary()

In [ ]:
# Entrenamiento del modelo y ploteo de las métricas

history = modelo.fit(P, T_OH, epochs=200, batch_size=16, verbose=True, validation_split=0.2)

plot_metric(history, "loss")
plot_metric(history, "accuracy")